# 导入函数库

In [1]:
import torch
import torchvision
from torchvision import models
import numpy as np
import torch.nn as nn

# 加载在ImageNet上预训练过的模型

In [2]:
model = models.vgg16(pretrained=True)

## 查看模型的结构

In [3]:
model.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size

## 查看模型的卷积层和全连接层的参数shape

In [4]:
for p in model.parameters():
    print(p.shape)

torch.Size([64, 3, 3, 3])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([256, 128, 3, 3])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([512, 256, 3, 3])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([4096, 25088])
torch.Size([4096])
torch.Size([4096, 4096])
torch.Size([4096])
torch.Size([1000, 4096])
torch.Size([1000])


## 参数个数统计

In [5]:
para = sum([np.prod(list(p.size())) for p in model.parameters()])  # np.prod([3, 4, 5]) = 60

## 计算参数大小，因为存储的是FP32，相当于4个字节

这里我们换算成熟悉的Mb表示，我们可以发现这个和实际下载过来的vgg16.pth的模型大小是一致的

In [6]:
para * 4 / 1024 / 1024

527.7921447753906

# 计算模型forward和backward过程中，产生的output feature map的大小

## 模拟输入，注意输入的shape要和预训练模型的输入shape一致

In [7]:
input = torch.ones([1, 3, 224, 224], dtype=torch.float32)
input_ = input.clone()
input_.requires_grad_(requires_grad=False)

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]])

## 把模型里的模块转换成列表形式

In [8]:
mods = list(model.modules())

## 收集输出的output feature map

注意这里要分2段来计算，我们可以输出mods来观察一下,可以发现有些模块不是我们想要的

In [9]:
mods

[VGG(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (1): ReLU(inplace=True)
     (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (3): ReLU(inplace=True)
     (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (6): ReLU(inplace=True)
     (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (8): ReLU(inplace=True)
     (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (11): ReLU(inplace=True)
     (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (13): ReLU(inplace=True)
     (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (15): ReLU(inplace=True)
     (16): MaxPool2d(kernel_size=2, stride=2, pa

In [10]:
out_sizes = []

## 第一段模块输出收集,这个是卷积模块,输出shape=[batch_size,C,H,W]

In [11]:
for i in range(2, 34):
    m = mods[i]
    if isinstance(m, nn.ReLU):
        # inplace=True means that it will modify the input directly, without allocating any additional
        # output. It can sometimes slightly decrease the memory usage, but may not always be a valid
        # operation(because the original input is destroyed). if you don't see an error, it means that
        # your use case is valid
        if m.inplace:
            continue
    out = m(input_)
    out_sizes.append(np.array(out.size()))
    input_ = out

In [12]:
input_.shape

torch.Size([1, 512, 7, 7])

## 卷积层的输出要转化成二维的

In [13]:
input_ = input_.view(-1, 512*7*7)

## 第二段模块输出收集，这个是全连接模块,输入shape=[batch_size,C * H * W]

In [14]:
for i in range(35, len(mods)):
    m = mods[i]
    if isinstance(m, nn.ReLU):
        # inplace=True means that it will modify the input directly, without allocating any additional
        # output. It can sometimes slightly decrease the memory usage, but may not always be a valid
        # operation(because the original input is destroyed). if you don't see an error, it means that
        # your use case is valid
        if m.inplace:
            continue
    out = m(input_)
    out_sizes.append(np.array(out.size()))
    input_ = out

## 展示收集到的中间变量

In [15]:
out_sizes

[array([  1,  64, 224, 224]),
 array([  1,  64, 224, 224]),
 array([  1,  64, 112, 112]),
 array([  1, 128, 112, 112]),
 array([  1, 128, 112, 112]),
 array([  1, 128,  56,  56]),
 array([  1, 256,  56,  56]),
 array([  1, 256,  56,  56]),
 array([  1, 256,  56,  56]),
 array([  1, 256,  28,  28]),
 array([  1, 512,  28,  28]),
 array([  1, 512,  28,  28]),
 array([  1, 512,  28,  28]),
 array([  1, 512,  14,  14]),
 array([  1, 512,  14,  14]),
 array([  1, 512,  14,  14]),
 array([  1, 512,  14,  14]),
 array([  1, 512,   7,   7]),
 array([  1, 512,   7,   7]),
 array([   1, 4096]),
 array([   1, 4096]),
 array([   1, 4096]),
 array([   1, 4096]),
 array([   1, 1000])]

## 统计中间输出变量的个数

In [16]:
total_nums = 0
for i in range(len(out_sizes)):
    s = out_sizes[i]
    nums = np.prod(np.array(s))
    total_nums += nums

In [17]:
total_nums

15120360

## 转化成熟悉的Mb表示

In [18]:
print("Model {} : intermedite variables: {:.3f} M (without backward)".format(model._get_name(), total_nums * 4 / 1000 /1000))
print("Model {} : intermedite variables: {:.3f} M (with backward)".format(model._get_name(), total_nums * 2 * 4 / 1000 / 1000))

Model VGG : intermedite variables: 60.481 M (without backward)
Model VGG : intermedite variables: 120.963 M (with backward)
